# Automação de processos Web

Comercialização de produtos influenciados pela cotação de moedas estrangeiras e ouro.

É preciso fazer a atualização diária dessas cotações para captar o valor de venda dos produtos.

In [ ]:
#instalar o pacote selenium caso não exista
#pip install -U pip
#pip install -U selenium

In [33]:
from selenium import webdriver

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

navegador = webdriver.Chrome()

#Cotação do dólar
navegador.get("https://www.google.com/")
navegador.find_element(By.XPATH, "/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input").send_keys("cotação dólar", Keys.ENTER)
cotacao_dolar = navegador.find_element(By.XPATH, '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value")

#Cotação do euro
navegador.get("https://www.google.com/")
navegador.find_element(By.XPATH, "/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input").send_keys("cotação euro", Keys.ENTER)
cotacao_euro = navegador.find_element(By.XPATH, '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value")

#Cotação do ouro
navegador.get("https://www.melhorcambio.com/ouro-hoje")
cotacao_ouro = navegador.find_element(By.XPATH, '//*[@id="comercial"]').get_attribute("value")
cotacao_ouro = cotacao_ouro.replace(",",".")

navegador.quit()


In [34]:
import pandas as pd

#Realização de cálculos na planilha após coleta de cotações
produtos = pd.read_excel('Produtos.xlsx')

display(produtos)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


In [35]:
produtos.loc[produtos['Moeda'] == "Dólar", 'Cotação'] = float(cotacao_dolar)
produtos.loc[produtos['Moeda'] == "Euro", 'Cotação'] = float(cotacao_euro)
produtos.loc[produtos['Moeda'] == "Ouro", 'Cotação'] = float(cotacao_ouro)

produtos["Preço de Compra"] = produtos["Preço Original"] * produtos["Cotação"]
produtos["Preço de Venda"] = produtos["Preço de Compra"] * produtos["Margem"]

display(produtos)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.4894,5489.345106,1.40,7685.083148
1,Carro Renault,4500.00,Euro,6.1962,27882.900000,2.00,55765.800000
2,Notebook Dell,899.99,Dólar,5.4894,4940.405106,1.70,8398.688680
3,IPhone,799.00,Dólar,5.4894,4386.030600,1.70,7456.252020
4,Carro Fiat,3000.00,Euro,6.1962,18588.600000,1.90,35318.340000
5,Celular Xiaomi,480.48,Dólar,5.4894,2637.546912,2.00,5275.093824
6,Joia 20g,20.00,Ouro,325.9300,6518.600000,1.15,7496.390000


In [36]:
#Exportar o resultado para outra planilha

produtos.to_excel("Produtos_Cotados.xlsx", index = False)